## Neural Network

using code fro

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

There are 4 csv files in the current version of the dataset:

In [2]:
df1 = pd.read_csv('./Dataset/user_a.csv', delimiter=',')
df2 = pd.read_csv('./Dataset/user_b.csv', delimiter=',')
df3 = pd.read_csv('./Dataset/user_c.csv', delimiter=',')
df4 = pd.read_csv('./Dataset/user_d.csv', delimiter=',')


=================================================================

## The first thing we will do is import all the libraries

We will be using the lower resolution MINST data set

In [3]:
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import confusion_matrix, accuracy_score # 
import numpy as np
import numpy.random as r # We will randomly initialize our weights
import matplotlib.pyplot as plt




In [ ]:
from sklearn.neural_network import MLPClassifier

def run(X, y, layers, test_size=0.4, lam=0.001, a=0.25, iter_num=500, batch_size='auto'):
    X_scale = StandardScaler()
    X = X_scale.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    clf = MLPClassifier(hidden_layer_sizes=layers, 
                        alpha=lam, # refers to our lambda
                        learning_rate_init=a, # refers to our alpha
                        max_iter=iter_num,
                        batch_size=batch_size, # default is 200
                        solver='sgd', 
                        random_state=1,
                        verbose=False
                       )
    clf.fit(X_train, y_train)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)

    if test_acc > 0.8:
        print("* * *")
        print(f"training accuracy: {train_acc*100}%")
        print(f"testing accuracy: {test_acc*100}%")

        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        print (f"\nConfusion Matrix: \n{cm}")

        print("\nfirst 10 test values and predictions:")

        # show predictions of first 10 test examples
        n = 10
        y_pred_n = clf.predict_proba(X_test[:n, :])
        for i in range(n):
            print(y_test[i], y_pred_n[i])
    
    return train_acc, test_acc

model_info = {} # key is accuracy, values are 
        
for name, X, y in (('all features', df1.values[:, 1:], df1.values[:, 0].astype(int)), 
                   ('std features', df1.values[:, 1::2], df1.values[:, 0].astype(int)), 
                   ('mean features', df1.values[:, 2::2], df1.values[:, 0].astype(int))
                  ):
    for layers in ((112, 56, 30, 10),
                   (112),
                   (112, 112, 112)
                  ):
        for test_size in (0.4, 0.3, 0.2):
            for alpha in (1., 0.1, 0.01, 0.001):
                for lam in (1000, 100, 1., 0.1, 0.01, 0.001, 0.0001, 0.00001):
                    print('\n==============================\n')
                    print(f'{name}:\nlayers={layers}\ntest_size={test_size}\nalpha={alpha}\nlambda={lam}\n')
                    tr_acc, ts_acc = run(X, y, layers, test_size, lam, alpha)
                    model_info[(tr_acc, ts_acc)] = 
                      {
                          'name' : name,
                          'layers' : layers,
                          'test size' : test_size,
                          'alpha' : alpha,
                          'lambda' : lam
                      }


all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



/home/el/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/el/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/el/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(



* * *

training accuracy: 33.62268518518518%
testing accuracy: 32.89930555555556%

Confusion Matrix: 
[[379   0   0]
 [401   0   0]
 [372   0   0]]

first 10 test values and predictions:
0 [nan nan nan]
1 [nan nan nan]
0 [nan nan nan]
2 [nan nan nan]
1 [nan nan nan]
2 [nan nan nan]
1 [nan nan nan]
1 [nan nan nan]
1 [nan nan nan]
0 [nan nan nan]
all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100


* * *

training accuracy: 33.449074074074076%
testing accuracy: 33.15972222222222%

Confusion Matrix: 
[[  0   0 372]
 [  0   0 398]
 [  0   0 382]]

first 10 test values and predictions:
2 [0.28922673 0.32789317 0.3828801 ]
2 [0.28922673 0.32789317 0.3828801 ]
2 [0.28922673 0.32789317 0.3828801 ]
1 [0.28922673 0.32789317 0.3828801 ]
0 [0.28922673 0.32789317 0.3828801 ]
1 [0.28922673 0.32789317 0.3828801 ]
0 [0.28922673 0.32789317 0.3828801 ]
2 [0.28922673 0.32789317 0.3828801 ]
2 [0.28922673 0.32789317 0.3828801 ]
1 [0.28922673 0.32789317 0.3828801 ]
all features:
laye


* * *

training accuracy: 100.0%
testing accuracy: 84.89583333333334%

Confusion Matrix: 
[[336  33  20]
 [ 28 314  34]
 [ 27  32 328]]

first 10 test values and predictions:
2 [9.53815767e-01 4.42259074e-08 4.61841891e-02]
0 [9.99739471e-01 2.41785336e-04 1.87435708e-05]
1 [4.01144853e-06 9.99995901e-01 8.74204200e-08]
1 [2.62419991e-07 9.99999738e-01 1.84560449e-11]
2 [2.72797747e-08 7.37300319e-06 9.99992600e-01]
0 [0.43731436 0.00083015 0.56185549]
2 [4.09058932e-03 2.51728320e-04 9.95657682e-01]
2 [5.10400791e-07 5.86134645e-07 9.99998903e-01]
2 [8.68312835e-07 5.01250890e-06 9.99994119e-01]
0 [1.35419598e-03 9.22664131e-07 9.98644881e-01]
all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.0001


* * *

training accuracy: 99.30555555555556%
testing accuracy: 81.16319444444444%

Confusion Matrix: 
[[342  41  20]
 [ 32 308  34]
 [ 33  57 285]]

first 10 test values and predictions:
2 [1.39810994e-06 1.61442295e-04 9.99837160e-01]
2 [0.54539398 0.43163879 0.0229

/home/el/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


...


* * *

training accuracy: 33.449074074074076%
testing accuracy: 33.15972222222222%

Confusion Matrix: 
[[382   0   0]
 [379   0   0]
 [391   0   0]]

first 10 test values and predictions:
1 [nan nan nan]
2 [nan nan nan]
2 [nan nan nan]
1 [nan nan nan]
2 [nan nan nan]
0 [nan nan nan]
0 [nan nan nan]
2 [nan nan nan]
1 [nan nan nan]
2 [nan nan nan]
std features:
layers=(112, 112, 112)
test_size=0.3
alpha=1.0
lambda=1e-05


* * *

training accuracy: 33.101851851851855%
testing accuracy: 33.68055555555556%

Confusion Matrix: 
[[  0 386   0]
 [  0 388   0]
 [  0 378   0]]

first 10 test values and predictions:
0 [0.33641762 0.34506245 0.31851993]
0 [0.33641762 0.34506245 0.31851993]
2 [0.33641762 0.34506245 0.31851993]
1 [0.33641762 0.34506245 0.31851993]
2 [0.33641762 0.34506245 0.31851993]
1 [0.33641762 0.34506245 0.31851993]
2 [0.33641762 0.34506245 0.31851993]
1 [0.33641762 0.34506245 0.31851993]
0 [0.33641762 0.34506245 0.31851993]
1 [0.33641762 0.34506245 0.31851993]
std features:
lay

/home/el/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(



* * *

training accuracy: 100.0%
testing accuracy: 77.77777777777779%

Confusion Matrix: 
[[317  37  49]
 [ 47 286  59]
 [ 24  40 293]]

first 10 test values and predictions:
0 [0.82865872 0.06602029 0.10532099]
1 [8.89007907e-01 1.10298255e-01 6.93837739e-04]
1 [0.37596354 0.62014081 0.00389565]
2 [0.00627694 0.00636591 0.98735715]
0 [9.99980849e-01 2.73141757e-07 1.88777224e-05]
2 [0.05443588 0.08098096 0.86458316]
0 [0.67188072 0.00260578 0.3255135 ]
0 [0.96303443 0.00658843 0.03037714]
1 [6.04704663e-05 9.99807487e-01 1.32042738e-04]
2 [0.00547899 0.00254291 0.99197811]
std features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.1


* * *

training accuracy: 100.0%
testing accuracy: 73.87152777777779%

Confusion Matrix: 
[[276  59  33]
 [ 48 290  68]
 [ 31  62 285]]

first 10 test values and predictions:
0 [1.00000000e+00 9.21284965e-15 3.67227067e-12]
0 [9.99999422e-01 5.56871892e-07 2.09414055e-08]
0 [1.00000000e+00 4.42076527e-14 4.72059080e-26]
1 [1.26007671e-03 9.9

/home/el/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(



* * *

training accuracy: 98.37962962962963%
testing accuracy: 75.26041666666666%

Confusion Matrix: 
[[301  43  43]
 [ 45 283  62]
 [ 33  59 283]]

first 10 test values and predictions:
2 [0.23560674 0.04304255 0.72135071]
0 [0.8202071  0.06957573 0.11021717]
2 [0.00833987 0.95677615 0.03488398]
2 [0.38438537 0.20519409 0.41042054]
1 [0.00202764 0.99304822 0.00492414]
0 [9.97932961e-01 1.95145469e-03 1.15584475e-04]
0 [9.93494609e-01 8.51928389e-04 5.65346283e-03]
2 [5.83866419e-04 3.69746983e-02 9.62441435e-01]
2 [0.16162436 0.0012569  0.83711874]
1 [3.43022718e-04 9.95454613e-01 4.20236400e-03]
std features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.1



Okay, so that took really long and was not that great. I think some more preprocessing needs to happen so that we aren't entering in 128 features.

- update: going from 30 in hidden layer to 66 drastically improved performance, from ~26% to ~39.5%
- and the neurons were all guessing 0 before, but now they're more consistently guessing other values. Although, 2 is not accounted for well.


...also, it looks like the neurons are almost always guessing 0?

### With Regularization

## to try
- using the means as features
- using just the variances as features
- changing the sample size
- changing the activation function
- changing number of hidden layers